In [ ]:
# Environment setup #base(python 3.8.6)
import pandas as pd
import re
import xlrd
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import gensim
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import os

In [ ]:
# Replace 'your_directory_path' with the path of your desired directory
directory_path = 'G:/academic/'

# Change the working directory to the specified directory path
os.chdir(directory_path)

# Verify that the working directory has been changed by printing the current directory
print(os.getcwd())

In [ ]:
# Replace 'your_excel_file.xlsx' with the name or path of your Excel file
file_path = 'data/data_name.xlsx'

# Read the Excel file and load it into a DataFrame
df = pd.read_excel(file_path, engine='openpyxl')

# print("null exist:", df.isnull().values.any()) # Null check
# df = df.dropna(how = 'any') # Null delete
# print("null exist(renew):", df.isnull().values.any()) # Null re-check
print("number of title:", len(df)) # counting title

# Display the first few rows of the DataFrame
df.head()


In [ ]:
# Extract the 'title' and 'company' columns as lists
title = df['title'].tolist()
company = df['company'].tolist()
date = df['date'].tolist()

# lexicon setting
with open("code/pos_pol_word.txt", "r", encoding="UTF-8") as file:
    positive = [line.strip() for line in file.readlines()[1:] if line.strip()]
    
with open("code/neg_pol_word_apartment.txt", "r", encoding="UTF-8") as file:
    negative = [line.strip() for line in file.readlines()[1:] if line.strip()]

# Initialize the Okt tokenizer (for Korean Language)
okt = Okt()

In [ ]:
## Sentiment function setting (Word Matching Added)

def sentiment_score(title, positive, negative):
    words = okt.morphs(title, norm=True)  # Tokenize Korean text using Okt

    pos_matches = [word for word in words if word in positive]  # Find words in 'words' that are in 'positive'
    neg_matches = [word for word in words if word in negative]

    score = len(pos_matches) - len(neg_matches)  # pos-neg
    pos_count = len(pos_matches)
    neg_count = len(neg_matches)

    return score, pos_count, neg_count, pos_matches, neg_matches

def sentimental(title, company, positive, negative):
    scores_and_counts = pd.Series(title).apply(lambda x: sentiment_score(x, positive, negative))
    scores = [score for score, _, _, _, _ in scores_and_counts]
    pos_counts = [pos_count for _, pos_count, _, _, _ in scores_and_counts]
    neg_counts = [neg_count for _, _, neg_count, _, _ in scores_and_counts]
    pos_matches = [pos_match for _, _, _, pos_match, _ in scores_and_counts]
    neg_matches = [neg_match for _, _, _, _, neg_match in scores_and_counts]

    scores_df = pd.DataFrame({'date': date, 'company': company, 'senti_score': scores, 'positive_count': pos_counts, 'negative_count': neg_counts, 'positive_matches': pos_matches, 'negative_matches': neg_matches, 'title': title})
    return scores_df

# Use the sentimental() function and save the result in the 'result' variable
result = sentimental(title, company, positive, negative)

# Save the DataFrame 'result' to an Excel file
result.to_excel("wordsum_match_apartment.xlsx", index=False, engine="openpyxl")
